In [1]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Import core agent functionality for creating AI agents and running them
from agents import Agent, Runner, Trace, function_tool, trace

from agents.extensions.visualization import draw_graph

# Import OpenAI response types for handling streaming responses
from openai.types.responses import ResponseTextDeltaEvent

# Import type hints for better code documentation and IDE support
from typing import Dict, List, Optional

# Import SendGrid for email functionality
import sendgrid
import os

# Import SendGrid email components for creating and sending emails
from sendgrid.helpers.mail import Mail, Email, To, Content

import asyncio

import pandas as pd

from datetime import datetime

from pydantic import BaseModel, Field
from openai import OpenAI

import json

from typing import List, Union


from dataclasses import asdict

print("Successfully imported all dependencies")


Successfully imported all dependencies


In [2]:
load_dotenv(override=True)


openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"OPENAI_API_KEY found. First 4 chars: {openai_api_key[:4]}")
    print("OpenAI API key was successfully found.")
else:
    print("OPENAI_API_KEY not found in environment variables.")










OPENAI_API_KEY found. First 4 chars: sk-p
OpenAI API key was successfully found.


In [3]:
# Read the CSV file containing active promotions
def read_active_promotions(csv_file_path: str = r"C:\Users\javon\projects\agents\TrueActivePromos.csv") -> pd.DataFrame:
    """
    Read the ActivePromotions.csv file and return a pandas DataFrame.
    
    Args:
        csv_file_path (str): Path to the CSV file. Defaults to the full path to ActivePromotions.csv
    
    Returns:
        pd.DataFrame: DataFrame containing the active promotions data
    """
    try:
        df = pd.read_csv(csv_file_path)
        print(f"Successfully loaded {len(df)} active promotions from {csv_file_path}")
        return df
    except FileNotFoundError:
        print(f"Error: File '{csv_file_path}' not found.")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return pd.DataFrame()

# Example usage - you can call this function to load the promotions data
# promotions_df = read_active_promotions()


In [4]:
# Read the CSV file containing active promotions
def read_active_promotions(csv_file_path: str = r"C:\Users\javon\projects\agents\TrueActivePromos.csv") -> pd.DataFrame:
    """
    Read the ActivePromotions.csv file and return a pandas DataFrame.
    
    Args:
        csv_file_path (str): Path to the CSV file. Defaults to the full path to ActivePromotions.csv
    
    Returns:
        pd.DataFrame: DataFrame containing the active promotions data
    """
    try:
        df = pd.read_csv(csv_file_path)
        print(f"Successfully loaded {len(df)} active promotions from {csv_file_path}")
        return df
    except FileNotFoundError:
        print(f"Error: File '{csv_file_path}' not found.")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return pd.DataFrame()

# Example usage - you can call this function to load the promotions data
# promotions_df = read_active_promotions()


In [5]:
from pydantic import BaseModel, Field

class BriefCreator(BaseModel):
    campaign_name: str = Field(..., description="The name of the campaign")
    campaign_date: str = Field(..., description="The date of the campaign")
    campaign_subject_line: str = Field(..., description="The subject line of the campaign")
    campaign_preheader: str = Field(..., description="The preheader of the campaign")
    campaign_objective: str = Field(..., description="The objective of the campaign")
    sale1_name: str = Field(..., description="The name of the first sale")
    sale2_name: str = Field(..., description="The name of the second sale")
    sale3_name: str = Field(..., description="The name of the third sale")
    sale1_image_description: str = Field(..., description="The description of the first sale image")
    sale1_cta: str = Field(..., description="The call to action of the first sale")
    sale2_image_description: str = Field(..., description="The description of the second sale image")
    sale2_cta: str = Field(..., description="The call to action of the second sale")
    sale3_image_description: str = Field(..., description="The description of the third sale image")
    sale3_cta: str = Field(..., description="The call to action of the third sale")

In [6]:
# Tool exposed to the agent
@function_tool
async def search_promotions_by_date(target_date: str) -> dict:
    """
    Agent tool: Search for active promotions on a specific date.
    """
    try:
        # Load data using your reliable function
        df = read_active_promotions(r"C:\Users\javon\projects\agents\TrueActivePromos.csv")
        if df.empty:
            return {"error": "CSV file could not be loaded."}

        # Date conversion
        target_dt = datetime.strptime(target_date, '%m/%d/%Y' if '/' in target_date else '%Y-%m-%d')
        df['StartDate'] = pd.to_datetime(df['StartDate'], format='%m/%d/%Y')
        df['EndDate'] = pd.to_datetime(df['EndDate'], format='%m/%d/%Y')

        # Filter for active
        active = df[
            (df['StartDate'] <= target_dt) &
            (df['EndDate'] >= target_dt) &
            (df['Status'].str.lower() == 'active')
        ]

        # Format results
        output = active[[
    'Name', 'StartDate', 'EndDate', 'URL',
    'Status', 'nameForPods', 'Blurb'
]].copy()

        output['StartDate'] = output['StartDate'].dt.strftime('%Y-%m-%d')
        output['EndDate'] = output['EndDate'].dt.strftime('%Y-%m-%d')

        return {"promotions": output.to_dict(orient="records")}

    except Exception as e:
        return {"error": str(e)}

        
analyst_tools = [search_promotions_by_date]


In [7]:

from dataclasses import field


class PromotionReturn(BaseModel):
    name: str = Field(..., description="The name of the promotion")
    start_date: str = Field(..., description="The start date of the promotion")
    end_date: str = Field(..., description="The end date of the promotion")
    #url: str = Field(..., description="The URL of the promotion")
    status: str = Field(..., description="The status of the promotion")
    name_for_pods: str
    blurb: str

class PromotionsOutput(BaseModel):
    promotions: List[PromotionReturn] = Field(..., description="The list of promotions")


In [8]:
Promotion_Agent_Instructions = """
You are an assistant whose responsibility is to return active promotions for a given date.
When a user provides a date, search the available promotions and return all promotions that are active on that date.
If no promotions are found for the given date, politely inform the user.
Be concise and accurate in your responses.
Return the results as a JSON object (dictionary) with a key 'promotions' containing a list of promotions, 
where each promotion is structured according to the PromotionReturn class fields.
"""

from typing import Dict, List

# The output_type is now a dictionary with a key 'promotions' containing a list of PromotionReturn objects
promotion_agent = Agent(
    name="Promotion Agent",
    instructions=Promotion_Agent_Instructions,
    model="gpt-4o-mini",
    tools=analyst_tools,
    output_type=PromotionsOutput  # 👈 Key addition
)

promotion_agent_as_tool = promotion_agent.as_tool(tool_name="promotion_agent", tool_description="Return active promotions for a given date")









In [14]:
EMAIL_MANAGER_INSTRUCTIONS = """
You are the Email Manager, responsible for creating compelling email stories based on active Shoe Carnival promotions. You are called when a user wants a themed email, a campaign brief, or a structured promotional story.

Responsibilities:
1. Retrieve Active Promotions:
   - Use the available tool to fetch all active promotions for the requested date.
   - Only use real promotions from this tool — never invent.

2. Filter & Select Promotions:
   - Based on the user's request (e.g., "athletics", "back to school"), filter promotions by looking at the `name`, `nameForPods`, or `blurb` fields.
   - Select 2–3 that best match the theme or strategy and explain why they were chosen.

3. Create a Campaign Story:
   - Create a compelling, cohesive promotional narrative using the selected promotions.
   - Maintain Shoe Carnival's brand voice: friendly, clear, family-focused, and value-driven.
   - Clearly communicate the campaign’s objective and why these promotions work well together.


Your output should be professional, structured, and immediately useful for building an email.

You are the final step before a campaign brief is ready for creative or deployment.
"""


email_manager_tools = [promotion_agent_as_tool]

Email_Manager = Agent(
    name="Email Manager",
    instructions=Email_Manager_Instructions,
    model="gpt-4o-mini",
    tools=email_manager_tools,
    handoff_description="Handoff to the Email Manager to handle promotion retrieval and email story creation"
)



In [22]:
#Sample code for promotion retrieval

result = await Runner.run(promotion_agent, input="What promotions are active on 07/19/2025?")

print(result.final_output)

Successfully loaded 45 active promotions from C:\Users\javon\projects\agents\TrueActivePromos.csv
promotions=[PromotionReturn(name='BOGOF SANDALS', start_date='2025-04-16', end_date='2025-07-22', url='', status='Active', name_for_pods='BOGOF SANDALS (04/16/2025-07/22/2025)', blurb='In Store and Online\nBOGOF SANDALS\nShop Now'), PromotionReturn(name='50% OFF SELECT W/K SANDALS MIC & CART PRICING', start_date='2025-06-25', end_date='2025-08-05', url='https://www.shoecarnival.com/flash-sale-4', status='Active', name_for_pods='50% OFF SELECT W/K SANDALS MIC & CART PRICING (06/25/2025-08/05/2025)', blurb='In Store and Online\n50% OFF SELECT W/K SANDALS MIC & CART PRICING\nShop Now'), PromotionReturn(name='FAMILY SANDAL SALE STARTING AT $19.98 MIC & CART PRICING', start_date='2025-06-25', end_date='2025-08-05', url='https://www.shoecarnival.com/family-sandals?sale=true', status='Active', name_for_pods='FAMILY SANDAL SALE STARTING AT $19.98 MIC & CART PRICING (06/25/2025-08/05/2025)', blurb=

In [13]:
CONVERSATIONAL_AGENT_INSTRUCTIONS = """
You are an AI marketing assistant for Shoe Carnival. Your primary role is to help users explore active promotions and guide them toward creating effective marketing campaign briefs — by routing requests to the correct agent.

Capabilities:
- You have access to one tool: `promotion_agent_as_tool`. This tool returns all active promotions for a given date.
- You also have access to the Email Manager agent, who can create full campaign stories and handle themed briefs.

What You Can Do:
- Answer casual questions about what promotions are active on a given date.
  - Use the promotion tool to retrieve and clearly list active promotions.
- If the user requests a themed campaign (e.g., "Make an athletics email for 07/19/2025") or asks for a full email idea or brief:
  - Do **not** create a story yourself.
  - Instead, **handoff to the Email Manager**, who will analyze promotions, select the most relevant ones, and generate a campaign story.
- If the user is unsure, offer helpful suggestions (e.g., popular themes like sandals, back-to-school, athletics) and invite them to pick one to proceed.

Limitations:
- Never create a full campaign story or brief — that is the Email Manager’s job.
- Never invent or reference promotions that do not come from the promotion tool.

Your tone should be friendly, helpful, and proactive in getting the user what they need — either directly or by routing to the right agent.
"""




conversational_tools = [promotion_agent_as_tool]
handoffs = [Email_Manager]


conversational_agent = Agent(
    name="Conversational Agent",
    instructions=CONVERSATIONAL_AGENT_INSTRUCTIONS,
    model="gpt-4o-mini",
    handoffs=handoffs,
)

In [26]:
import gradio as gr
from typing import List, Dict

# Initial context
initial_context = {
    "last_promotions": [],
    "last_theme_request": None,
    "last_requested_date": None,
    "last_custom_input": ""
}


async def chat_with_agent(message: str, history: List[List[str]], context: dict) -> tuple[str, List[List[str]], dict]:
    try:
        context["last_custom_input"] = message

        # Build recent conversational context
        convo = ""
        for user_msg, assistant_msg in history[-5:]:  # Keep last 5 turns
            convo += f"User: {user_msg}\nAssistant: {assistant_msg}\n"
        convo += f"User: {message}\nAssistant:"

        # Pass combined input into the agent
        result = await Runner.run(conversational_agent, input=convo, context=context)

        # Handle tool result (your promotion tool output)
        if hasattr(result.final_output, 'promotions'):
            context["last_promotions"] = result.final_output.promotions

            response = f"Found {len(result.final_output.promotions)} active promotions:\n\n"
            for i, promo in enumerate(result.final_output.promotions, 1):
                response += f"{i}. {promo.name}\n"
                response += f"   Date Range: {promo.start_date} - {promo.end_date}\n"
                response += f"   Status: {promo.status}\n\n"
        else:
            response = str(result.final_output)

    except Exception as e:
        response = f"Error: {str(e)}"

    # Update visual + backend memory
    history.append([message, response])
    return "", history, context




# Create the Gradio interface
with gr.Blocks(title="Shoe Carnival Promotion Agent") as demo:
    gr.Markdown("# 🛍️ Shoe Carnival Promotion Agent")
    gr.Markdown("Ask about active promotions for any date!")

    chatbot = gr.Chatbot(label="Conversation History", height=400)
    msg = gr.Textbox(label="Your Message", placeholder="e.g., Create an athletics email for 07/19/2025")

    # 🧠 This keeps the Python dict between submissions
    memory = gr.State(initial_context)

    send_btn = gr.Button("Send")
    clear_btn = gr.Button("Clear History")

    # Set up events
    send_btn.click(
        fn=chat_with_agent,
        inputs=[msg, chatbot, memory],
        outputs=[msg, chatbot, memory]
    )

    msg.submit(
        fn=chat_with_agent,
        inputs=[msg, chatbot, memory],
        outputs=[msg, chatbot, memory]
    )

    clear_btn.click(
        fn=lambda: ([], "", initial_context),
        outputs=[chatbot, msg, memory]
    )


C:\Users\javon\AppData\Local\Temp\ipykernel_32784\2837719034.py:53: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation History", height=400)


In [27]:

# Launch the app
if __name__ == "__main__":
    demo.launch(share=False, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Successfully loaded 45 active promotions from C:\Users\javon\projects\agents\TrueActivePromos.csv
Keyboard interruption in main thread... closing server.
